In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Oct 25 14:07:29 2021

@author: chumache
"""
import os
import json
import numpy as np
import torch
from torch import nn, optim
from torch.optim import lr_scheduler

from opts import parse_opts
from model import generate_model
import transforms 
from dataset import get_training_set, get_validation_set, get_test_set
from utils import Logger, adjust_learning_rate, save_checkpoint
from train import train_epoch
from validation import val_epoch
import time


if __name__ == '__main__':
    opt = parse_opts()
    n_folds = 1
    test_accuracies = []
    
    if opt.device != 'cpu':
        opt.device = 'cuda' if torch.cuda.is_available() else 'cpu'  

    pretrained = opt.pretrain_path != 'None'    
    
    #opt.result_path = 'res_'+str(time.time())
    if not os.path.exists(opt.result_path):
        os.makedirs(opt.result_path)
        
    opt.arch = '{}'.format(opt.model)  
    opt.store_name = '_'.join([opt.dataset, opt.model, str(opt.sample_duration)])
            
    for fold in range(1):
        #if opt.dataset == 'RAVDESS':
        #    opt.annotation_path = '/lustre/scratch/chumache/ravdess-develop/annotations_croppad_fold'+str(fold+1)+'.txt'

        print(opt)
        with open(os.path.join(opt.result_path, 'opts'+str(time.time())+str(fold)+'.json'), 'w') as opt_file:
            json.dump(vars(opt), opt_file)
            
        torch.manual_seed(opt.manual_seed)
        model, parameters = generate_model(opt)

        criterion = nn.CrossEntropyLoss()
        criterion = criterion.to(opt.device)
        
        if not opt.no_train:
            
            video_transform = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotate(),
                transforms.ToTensor(opt.video_norm_value)])
        
            training_data = get_training_set(opt, spatial_transform=video_transform) 
        
            train_loader = torch.utils.data.DataLoader(
                training_data,
                batch_size=opt.batch_size,
                shuffle=True,
                num_workers=0,
                pin_memory=False)
        
            train_logger = Logger(
                os.path.join(opt.result_path, 'train'+str(fold)+'.log'),
                ['epoch', 'loss', 'prec1', 'prec5', 'lr'])
            train_batch_logger = Logger(
                os.path.join(opt.result_path, 'train_batch'+str(fold)+'.log'),
                ['epoch', 'batch', 'iter', 'loss', 'prec1', 'prec5', 'lr'])
            

            optimizer = optim.SGD(
                parameters,
                lr=opt.learning_rate,
                momentum=opt.momentum,
                dampening=opt.dampening,
                weight_decay=opt.weight_decay,
                nesterov=False)
            scheduler = lr_scheduler.ReduceLROnPlateau(
                optimizer, 'min', patience=opt.lr_patience)
            
        if not opt.no_val:
            video_transform = transforms.Compose([
                transforms.ToTensor(opt.video_norm_value)])     
        
            validation_data = get_validation_set(opt, spatial_transform=video_transform)
            
            val_loader = torch.utils.data.DataLoader(
                validation_data,
                batch_size=opt.batch_size,
                shuffle=False,
                num_workers=0,
                pin_memory=False)
        
            val_logger = Logger(
                    os.path.join(opt.result_path, 'val'+str(fold)+'.log'), ['epoch', 'loss', 'prec1', 'prec5'])
            test_logger = Logger(
                    os.path.join(opt.result_path, 'test'+str(fold)+'.log'), ['epoch', 'loss', 'prec1', 'prec5'])

            
        best_prec1 = 0
        best_loss = 1e10
        if opt.resume_path:
            print('loading checkpoint {}'.format(opt.resume_path))
            checkpoint = torch.load(opt.resume_path)
            assert opt.arch == checkpoint['arch']
            best_prec1 = checkpoint['best_prec1']
            opt.begin_epoch = checkpoint['epoch']
            model.load_state_dict(checkpoint['state_dict'])

        for i in range(opt.begin_epoch, opt.n_epochs + 1):
            print(i,"===============")
            if not opt.no_train:
                adjust_learning_rate(optimizer, i, opt)
                train_epoch(i, train_loader, model, criterion, optimizer, opt,
                            train_logger, train_batch_logger)
                state = {
                    'epoch': i,
                    'arch': opt.arch,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'best_prec1': best_prec1
                    }
                save_checkpoint(state, False, opt, fold)
            
            if not opt.no_val:
                
                validation_loss, prec1 = val_epoch(i, val_loader, model, criterion, opt,
                                            val_logger)
                is_best = prec1 > best_prec1
                best_prec1 = max(prec1, best_prec1)
                state = {
                'epoch': i,
                'arch': opt.arch,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'best_prec1': best_prec1
                }
               
                save_checkpoint(state, is_best, opt, fold)

               
        if opt.test:

            test_logger = Logger(
                    os.path.join(opt.result_path, 'test'+str(fold)+'.log'), ['epoch', 'loss', 'prec1', 'prec5'])

            video_transform = transforms.Compose([
                transforms.ToTensor(opt.video_norm_value)])
                
            test_data = get_test_set(opt, spatial_transform=video_transform) 
        
            #load best model
            best_state = torch.load('%s/%s_best' % (opt.result_path, opt.store_name)+str(fold)+'.pth')
            model.load_state_dict(best_state['state_dict'])
        
            test_loader = torch.utils.data.DataLoader(
                test_data,
                batch_size=opt.batch_size,
                shuffle=False,
                num_workers=opt.n_threads,
                pin_memory=True)
            
            test_loss, test_prec1 = val_epoch(10000, test_loader, model, criterion, opt,
                                            test_logger)
            
            with open(os.path.join(opt.result_path, 'test_set_bestval'+str(fold)+'.txt'), 'a') as f:
                    f.write('Prec1: ' + str(test_prec1) + '; Loss: ' + str(test_loss))
            test_accuracies.append(test_prec1) 
            
    with open(os.path.join(opt.result_path, 'test_set_bestval.txt'), 'a') as f:
        f.write('Prec1: ' + str(np.mean(np.array(test_accuracies))) +'+'+str(np.std(np.array(test_accuracies))) + '\n')


Namespace(phase='train', certification_method='randomized_ablation', k=3000, k1=5, k2=1000, annotation_path='ravdess_preprocessing/annotations.txt', result_path='results', store_name='RAVDESS_multimodalcnn_15', dataset='RAVDESS', n_classes=8, model='multimodalcnn', num_heads=1, device='cuda', sample_size=224, sample_duration=15, learning_rate=0.04, momentum=0.9, lr_steps=[30, 40, 65, 70, 200, 250], dampening=0.9, weight_decay=0.001, lr_patience=10, batch_size=8, n_epochs=50, begin_epoch=1, resume_path='', pretrain_path='Pretrained_EfficientFace.tar', no_train=False, no_val=False, test=True, test_subset='test', n_threads=1, video_norm_value=255, manual_seed=1, fusion='ia', mask='softhard', arch='multimodalcnn')
Initializing efficientnet
Total number of trainable parameters:  1854766
1 ===============
train at epoch 1
opt.model == multimodalcnn
train at epoch 1


C:\Users\dongs\Documents\multimodal-emotion-recognition-main\transforms.py:74: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))
C:\Users\dongs\Documents\multimodal-emotion-recognition-main\transforms.py:74: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))


Epoch: [1][0/240]	 lr: 0.04000	Time 5.442 (5.442)	Data 2.649 (2.649)	Loss 2.2072 (2.2072)	Prec@1 9.37500 (9.37500)	Prec@5 31.25000 (31.25000)
Epoch: [1][10/240]	 lr: 0.04000	Time 1.766 (2.107)	Data 1.527 (1.638)	Loss 1.8864 (2.2269)	Prec@1 28.12500 (11.36364)	Prec@5 87.50000 (58.80682)
Epoch: [1][20/240]	 lr: 0.04000	Time 1.835 (1.938)	Data 1.591 (1.584)	Loss 2.3589 (2.1748)	Prec@1 0.00000 (13.39286)	Prec@5 62.50000 (66.36905)
Epoch: [1][30/240]	 lr: 0.04000	Time 1.778 (1.868)	Data 1.543 (1.552)	Loss 2.1592 (2.1733)	Prec@1 9.37500 (14.41532)	Prec@5 50.00000 (64.51613)
Epoch: [1][40/240]	 lr: 0.04000	Time 1.741 (1.837)	Data 1.493 (1.542)	Loss 2.0989 (2.1548)	Prec@1 12.50000 (14.25305)	Prec@5 65.62500 (64.55792)
Epoch: [1][50/240]	 lr: 0.04000	Time 1.654 (1.807)	Data 1.419 (1.523)	Loss 2.0474 (2.1285)	Prec@1 9.37500 (13.78677)	Prec@5 75.00000 (66.91177)
Epoch: [1][60/240]	 lr: 0.04000	Time 1.731 (1.792)	Data 1.509 (1.517)	Loss 2.2388 (2.1108)	Prec@1 6.25000 (14.80533)	Prec@5 56.25000 (67

Epoch: [1][35/60]	Time 0.83438 (0.72167)	Data 0.80237 (0.68428)	Loss 2.5250 (1.6607)	Prec@1 0.00000 (42.14286)	Prec@5 100.00000 (93.21429)
Epoch: [1][36/60]	Time 0.66254 (0.72003)	Data 0.62653 (0.68268)	Loss 1.4985 (1.6562)	Prec@1 50.00000 (42.36111)	Prec@5 100.00000 (93.40278)
Epoch: [1][37/60]	Time 0.65256 (0.71821)	Data 0.61305 (0.68080)	Loss 0.8592 (1.6346)	Prec@1 50.00000 (42.56757)	Prec@5 100.00000 (93.58109)
Epoch: [1][38/60]	Time 0.64106 (0.71618)	Data 0.60706 (0.67885)	Loss 1.6753 (1.6357)	Prec@1 25.00000 (42.10526)	Prec@5 100.00000 (93.75000)
Epoch: [1][39/60]	Time 0.62811 (0.71392)	Data 0.60411 (0.67694)	Loss 2.2499 (1.6514)	Prec@1 25.00000 (41.66667)	Prec@5 87.50000 (93.58974)
Epoch: [1][40/60]	Time 0.76368 (0.71516)	Data 0.72816 (0.67822)	Loss 2.9707 (1.6844)	Prec@1 12.50000 (40.93750)	Prec@5 37.50000 (92.18750)
Epoch: [1][41/60]	Time 0.68370 (0.71439)	Data 0.64218 (0.67734)	Loss 2.3360 (1.7003)	Prec@1 12.50000 (40.24390)	Prec@5 100.00000 (92.37804)
Epoch: [1][42/60]	Time 

Epoch: [2][8/60]	Time 0.61577 (0.58373)	Data 0.58276 (0.54964)	Loss 1.4980 (1.0009)	Prec@1 50.00000 (62.50000)	Prec@5 100.00000 (100.00000)
Epoch: [2][9/60]	Time 0.65146 (0.59126)	Data 0.61691 (0.55711)	Loss 0.3521 (0.9288)	Prec@1 100.00000 (66.66666)	Prec@5 100.00000 (100.00000)
Epoch: [2][10/60]	Time 0.64606 (0.59674)	Data 0.60905 (0.56231)	Loss 0.9676 (0.9327)	Prec@1 75.00000 (67.50000)	Prec@5 100.00000 (100.00000)
Epoch: [2][11/60]	Time 0.70515 (0.60659)	Data 0.66714 (0.57184)	Loss 0.8927 (0.9290)	Prec@1 75.00000 (68.18182)	Prec@5 100.00000 (100.00000)
Epoch: [2][12/60]	Time 0.74090 (0.61778)	Data 0.69837 (0.58238)	Loss 0.8487 (0.9223)	Prec@1 75.00000 (68.75000)	Prec@5 100.00000 (100.00000)
Epoch: [2][13/60]	Time 0.72129 (0.62575)	Data 0.68228 (0.59007)	Loss 1.3242 (0.9532)	Prec@1 37.50000 (66.34615)	Prec@5 100.00000 (100.00000)
Epoch: [2][14/60]	Time 0.70671 (0.63153)	Data 0.67170 (0.59590)	Loss 0.0153 (0.8862)	Prec@1 100.00000 (68.75000)	Prec@5 100.00000 (100.00001)
Epoch: [2][15

...

Epoch: [48][38/60]	Time 0.58797 (0.62153)	Data 0.55396 (0.58577)	Loss 3.3260 (1.0511)	Prec@1 12.50000 (65.13158)	Prec@5 100.00000 (100.00000)
Epoch: [48][39/60]	Time 0.58507 (0.62060)	Data 0.55006 (0.58485)	Loss 0.4825 (1.0365)	Prec@1 87.50000 (65.70513)	Prec@5 100.00000 (100.00000)
Epoch: [48][40/60]	Time 0.64202 (0.62113)	Data 0.60401 (0.58533)	Loss 2.5082 (1.0733)	Prec@1 50.00000 (65.31250)	Prec@5 100.00000 (100.00000)
Epoch: [48][41/60]	Time 0.74631 (0.62419)	Data 0.70330 (0.58821)	Loss 1.5669 (1.0853)	Prec@1 50.00000 (64.93902)	Prec@5 100.00000 (100.00000)
Epoch: [48][42/60]	Time 0.70259 (0.62605)	Data 0.66358 (0.59001)	Loss 0.0160 (1.0599)	Prec@1 100.00000 (65.77381)	Prec@5 100.00000 (100.00000)
Epoch: [48][43/60]	Time 0.58601 (0.62512)	Data 0.54401 (0.58894)	Loss 0.1440 (1.0386)	Prec@1 87.50000 (66.27907)	Prec@5 100.00000 (100.00000)
Epoch: [48][44/60]	Time 0.57917 (0.62408)	Data 0.54212 (0.58787)	Loss 0.0034 (1.0151)	Prec@1 100.00000 (67.04546)	Prec@5 100.00000 (100.00000)
Epoc

Epoch: [49][10/60]	Time 0.62502 (0.58813)	Data 0.58901 (0.55353)	Loss 0.4770 (0.4524)	Prec@1 62.50000 (85.00000)	Prec@5 100.00000 (98.75000)
Epoch: [49][11/60]	Time 0.73928 (0.60188)	Data 0.70127 (0.56696)	Loss 1.3996 (0.5385)	Prec@1 62.50000 (82.95455)	Prec@5 100.00000 (98.86364)
Epoch: [49][12/60]	Time 0.73361 (0.61285)	Data 0.70161 (0.57818)	Loss 0.4825 (0.5339)	Prec@1 75.00000 (82.29167)	Prec@5 100.00000 (98.95834)
Epoch: [49][13/60]	Time 0.56302 (0.60902)	Data 0.53201 (0.57463)	Loss 0.0386 (0.4958)	Prec@1 100.00000 (83.65385)	Prec@5 100.00000 (99.03847)
Epoch: [49][14/60]	Time 0.59575 (0.60807)	Data 0.54727 (0.57267)	Loss 0.2121 (0.4755)	Prec@1 87.50000 (83.92857)	Prec@5 100.00000 (99.10715)
Epoch: [49][15/60]	Time 0.58795 (0.60673)	Data 0.54994 (0.57116)	Loss 1.0933 (0.5167)	Prec@1 50.00000 (81.66667)	Prec@5 87.50000 (98.33334)
Epoch: [49][16/60]	Time 0.59401 (0.60594)	Data 0.56300 (0.57065)	Loss 1.7968 (0.5967)	Prec@1 25.00000 (78.12500)	Prec@5 100.00000 (98.43750)
Epoch: [49][1

Epoch: [50][70/240]	 lr: 0.00040	Time 1.714 (1.809)	Data 1.457 (1.546)	Loss 0.6914 (0.7944)	Prec@1 78.12500 (70.02641)	Prec@5 96.87500 (94.45422)
Epoch: [50][80/240]	 lr: 0.00040	Time 1.978 (1.819)	Data 1.718 (1.555)	Loss 0.9061 (0.7874)	Prec@1 62.50000 (70.21605)	Prec@5 87.50000 (94.36729)
Epoch: [50][90/240]	 lr: 0.00040	Time 1.703 (1.823)	Data 1.440 (1.559)	Loss 0.7592 (0.7809)	Prec@1 78.12500 (70.60440)	Prec@5 93.75000 (94.33379)
Epoch: [50][100/240]	 lr: 0.00040	Time 1.738 (1.819)	Data 1.469 (1.555)	Loss 0.4998 (0.7697)	Prec@1 78.12500 (70.85396)	Prec@5 100.00000 (94.55445)
Epoch: [50][110/240]	 lr: 0.00040	Time 1.619 (1.818)	Data 1.359 (1.553)	Loss 0.9217 (0.7694)	Prec@1 62.50000 (71.00225)	Prec@5 87.50000 (94.76352)
Epoch: [50][120/240]	 lr: 0.00040	Time 1.878 (1.817)	Data 1.621 (1.552)	Loss 0.8307 (0.7653)	Prec@1 71.87500 (71.38429)	Prec@5 93.75000 (94.67975)
Epoch: [50][130/240]	 lr: 0.00040	Time 1.846 (1.819)	Data 1.575 (1.554)	Loss 0.7119 (0.7643)	Prec@1 75.00000 (71.49332)	

Epoch: [50][41/60]	Time 0.70249 (0.60782)	Data 0.66949 (0.57299)	Loss 2.8284 (0.9803)	Prec@1 25.00000 (68.59756)	Prec@5 100.00000 (99.39024)
Epoch: [50][42/60]	Time 0.56900 (0.60689)	Data 0.53599 (0.57211)	Loss 0.0248 (0.9576)	Prec@1 100.00000 (69.34524)	Prec@5 100.00000 (99.40476)
Epoch: [50][43/60]	Time 0.58003 (0.60627)	Data 0.54802 (0.57155)	Loss 0.1134 (0.9380)	Prec@1 87.50000 (69.76744)	Prec@5 100.00000 (99.41860)
Epoch: [50][44/60]	Time 0.56688 (0.60537)	Data 0.53487 (0.57071)	Loss 0.2589 (0.9225)	Prec@1 87.50000 (70.17046)	Prec@5 100.00000 (99.43182)
Epoch: [50][45/60]	Time 0.59802 (0.60521)	Data 0.56001 (0.57048)	Loss 1.5826 (0.9372)	Prec@1 37.50000 (69.44444)	Prec@5 100.00000 (99.44445)
Epoch: [50][46/60]	Time 0.70456 (0.60737)	Data 0.67055 (0.57265)	Loss 0.0735 (0.9184)	Prec@1 100.00000 (70.10870)	Prec@5 100.00000 (99.45652)
Epoch: [50][47/60]	Time 0.56910 (0.60656)	Data 0.53609 (0.57187)	Loss 0.0076 (0.8990)	Prec@1 100.00000 (70.74468)	Prec@5 100.00000 (99.46808)
Epoch: [50

Epoch: [10000][38/60]	Time 0.88558 (1.13757)	Data 0.84957 (1.09957)	Loss 0.5674 (0.8370)	Prec@1 62.50000 (71.05264)	Prec@5 100.00000 (99.67105)
Epoch: [10000][39/60]	Time 0.86882 (1.13068)	Data 0.83475 (1.09278)	Loss 1.5772 (0.8560)	Prec@1 50.00000 (70.51283)	Prec@5 100.00000 (99.67949)
Epoch: [10000][40/60]	Time 0.91999 (1.12541)	Data 0.87798 (1.08741)	Loss 0.6723 (0.8514)	Prec@1 75.00000 (70.62500)	Prec@5 100.00000 (99.68750)
Epoch: [10000][41/60]	Time 0.95139 (1.12117)	Data 0.91538 (1.08322)	Loss 0.1248 (0.8337)	Prec@1 87.50000 (71.03658)	Prec@5 100.00000 (99.69512)
Epoch: [10000][42/60]	Time 0.93915 (1.11683)	Data 0.89714 (1.07879)	Loss 0.7936 (0.8327)	Prec@1 87.50000 (71.42857)	Prec@5 87.50000 (99.40476)
Epoch: [10000][43/60]	Time 1.11033 (1.11668)	Data 1.07532 (1.07871)	Loss 2.7149 (0.8765)	Prec@1 37.50000 (70.63953)	Prec@5 75.00000 (98.83721)
Epoch: [10000][44/60]	Time 1.00778 (1.11421)	Data 0.96277 (1.07607)	Loss 0.0043 (0.8567)	Prec@1 100.00000 (71.30682)	Prec@5 100.00000 (98.